## 一、 2002 年-2018 年上海机动车拍照拍卖
### 问题
#### (1) 哪一次拍卖的中标率首次小于 5%？

In [2]:
import numpy as np
import pandas as pd

df = pd.read_csv('data/2002年-2018年上海机动车拍照拍卖.csv')
df['percent'] = df['Total number of license issued'] / df['Total number of applicants']
df.query('percent < 0.05')['Date'].iloc[0]

'15-May'

#### (2) 按年统计拍卖最低价的下列统计量：最大值、均值、 0.75 分位数，要求显示在同一张表上。

In [7]:
df.rename(columns={'lowest price ':'lowest price'}, inplace=True)
def g_func(i):
    return(int(df.iloc[i]['Date'].split(sep='-')[0]) + 2000)
df.groupby(g_func)['lowest price'].agg(['max', 'mean', lambda x:x.quantile(0.75)]) \
  .rename(columns={'max':'最大值', 'mean':'均值', '<lambda_0>':'0.75分位数'})

,最大值,均值,0.75分位数
2002,30800,20316.666667,24300
2003,38500,31983.333333,36300
2004,44200,29408.333333,38400
2005,37900,31908.333333,35600
2006,39900,37058.333333,39525
2007,53800,45691.666667,48950
2008,37300,29945.454545,34150
2009,36900,31333.333333,34150
2010,44900,38008.333333,41825
2011,53800,47958.333333,51000


#### (3) 将第一列时间列拆分成两个列，一列为年份（格式为 20××），另一列为月份（英语缩写），添加到列表作为第一第二列，并将原表第一列删除，其他列依次向后顺延。

In [120]:
df_tmp = df['Date'].str.split('-', expand=True)
df1 = pd.DataFrame({'Year':df_tmp.iloc[:, 0].astype('int64')+2000, 'Month':df_tmp.iloc[:, 1]}).join(df.drop(columns=['Date', 'percent']))
df1.head()

,Year,Month,Total number of license issued,lowest price,avg price,Total number of applicants
0,2002,Jan,1400,13600,14735,3718
1,2002,Feb,1800,13100,14057,4590
2,2002,Mar,2000,14300,14662,5190
3,2002,Apr,2300,16000,16334,4806
4,2002,May,2350,17800,18357,4665


#### (4) 现在将表格行索引设为多级索引，外层为年份，内层为原表格第二至第五列的变量名，列索引为月份。

In [128]:
df1.pivot(index='Month', columns='Year', values=list(df1.columns[2:6])).swaplevel(1, 0, axis=1)

Year,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
,Total number of license issued,Total number of license issued,Total number of license issued,Total number of license issued,Total number of license issued,Total number of license issued,Total number of license issued,Total number of license issued,Total number of license issued,Total number of license issued,...,Total number of applicants,Total number of applicants,Total number of applicants,Total number of applicants,Total number of applicants,Total number of applicants,Total number of applicants,Total number of applicants,Total number of applicants,Total number of applicants
Month,,,,,,,,,,,,,,,,,,,,,
Apr,2300.0,3300.0,5500.0,5000.0,5000.0,5500.0,9000.0,6500.0,8500.0,8000.0,...,17654.0,17313.0,22326.0,22706.0,26174.0,94241.0,152298.0,256897.0,252273.0,204980.0
Aug,3000.0,4500.0,6800.0,6829.0,6200.0,8000.0,6000.0,8000.0,9000.0,9000.0,...,18750.0,16855.0,21544.0,21425.0,22650.0,121550.0,166939.0,251188.0,256083.0,192755.0
Dec,3600.0,4776.0,5500.0,5700.0,6500.0,7500.0,4500.0,8000.0,9000.0,8500.0,...,18577.0,11224.0,26531.0,18244.0,39625.0,96972.0,179133.0,219882.0,228148.0,165442.0
Feb,1800.0,3000.0,4800.0,3800.0,3800.0,3500.0,NaN,5200.0,7500.0,7500.0,...,16848.0,18810.0,25104.0,23391.0,24651.0,45758.0,103224.0,196470.0,251717.0,220831.0
Jan,1400.0,3000.0,5000.0,5500.0,5000.0,6000.0,16000.0,5200.0,8000.0,8000.0,...,16544.0,18975.0,30675.0,24354.0,20857.0,41946.0,98203.0,187533.0,232101.0,226316.0
Jul,3000.0,6000.0,6600.0,6326.0,5500.0,5500.0,6800.0,8000.0,9000.0,9000.0,...,17220.0,13389.0,21852.0,26526.0,21811.0,136098.0,166302.0,240750.0,269189.0,202337.0
Jun,2800.0,5500.0,6233.0,5690.0,4500.0,6000.0,7700.0,8000.0,9200.0,9000.0,...,17433.0,16252.0,22474.0,24774.0,21482.0,135677.0,172205.0,275438.0,244349.0,209672.0
Mar,2000.0,3000.0,4800.0,4000.0,4500.0,4000.0,9300.0,6000.0,8000.0,8000.0,...,18575.0,17704.0,25014.0,24897.0,23589.0,61853.0,132690.0,221109.0,262010.0,217056.0


#### (5) 一般而言某个月最低价与上月最低价的差额，会与该月均值与上月均值的差额具有相同的正负号，哪些拍卖时间不具有这个特点？

In [18]:
l_lowest_price = df['lowest price'].tolist()
l_lowest_price_diff = []
pre = 0
for i in l_lowest_price:
    l_lowest_price_diff.append(1 if i >= pre else -1)
    pre = i

l_avg_price = df['avg price'].tolist()
l_avg_price_diff = []
pre = 0
for i in l_avg_price:
    l_avg_price_diff.append(1 if i >= pre else -1)
    pre = i

tmp_df = pd.DataFrame({'Date':df['Date'], 'low_avg_corr':(np.array(l_lowest_price_diff)+np.array(l_avg_price_diff))})
tmp_df.query('low_avg_corr == 0')['Date']

21      3-Oct
22      3-Nov
29      4-Jun
36      5-Jan
37      5-Feb
44      5-Sep
52      6-May
56      6-Sep
60      7-Jan
61      7-Feb
71      7-Dec
128    12-Oct
198    18-Aug
Name: Date, dtype: object

#### (6) 将某一个月牌照发行量与其前两个月发行量均值的差额定义为发行增益，最初的两个月用 0 填充，求发行增益极值出现的时间。

In [42]:
l_license = df['Total number of license issued'].tolist()
l_license_increase = [0, 0]
pre_befor_pre = l_license[0]
pre = l_license[1]
for i in l_license[2:]:
    l_license_increase.append(i - (pre_befor_pre + pre) / 2)
    pre_befor_pre = pre
    pre = i
tmp_df = pd.DataFrame({'Date':df['Date'], 'increate':l_license_increase})
print("max month:")
display(tmp_df.iloc[tmp_df['increate'].idxmax()]['Date'])
print("min month:")
display(tmp_df.iloc[tmp_df['increate'].idxmin()]['Date'])

max month:


'8-Jan'

min month:


'8-Apr'

## 二、 2007 年-2019 年俄罗斯机场货运航班运载量
### 问题
#### (1) 求每年货运航班总运量。

In [44]:
import numpy as np
import pandas as pd

df = pd.read_csv('data/2007年-2019年俄罗斯货运航班运载量.csv')
df.groupby('Year')['Whole year'].agg(sum)

Year
2007    659438.23
2008    664682.46
2009    560809.77
2010    693033.98
2011    818691.71
2012    846388.03
2013    792337.08
2014    729457.12
2015    630208.97
2016    679370.15
2017    773662.28
2018    767095.28
2019    764606.27
Name: Whole year, dtype: float64

#### (2) 每年记录的机场都是相同的吗？

In [66]:
same_record = True
tmp_df = pd.crosstab(index=df['Year'], columns=df['Airport name'])
for column in list(tmp_df.columns):
    if(tmp_df[column].nunique() != 1):
        same_record = False
        break
same_record

False

#### (3) 按年计算 2010 年-2015 年全年货运量记录为 0 的机场航班比例。

In [87]:
def func(sub_df):
    return(sub_df[sub_df['Whole year'] == 0.0].shape[0] / sub_df.shape[0])
df.query('Year in [2010,2011,2012,2013,2014,2015]').groupby('Year')[['Airport name', 'Whole year']].apply(func)

Year
2010    0.767123
2011    0.770548
2012    0.770548
2013    0.770548
2014    0.770548
2015    0.770548
dtype: float64

#### (4) 若某机场至少存在 5 年或以上满足所有月运量记录都为 0，则将其所有年份的记录信息从表中删除，并返回处理后的表格

In [109]:
s_tmp = df[df['Whole year'] == 0.0].groupby('Airport name')['Year'].agg(lambda x:x.count())
df[~df['Airport name'].isin(s_tmp[s_tmp >= 5].index.tolist())]

,Airport name,Year,January,February,March,April,May,June,July,August,September,October,November,December,Whole year,Airport coordinates
0,Abakan,2019,44.70,66.21,72.70,75.82,100.34,78.38,63.88,73.06,66.74,75.44,110.50,89.80,917.57,"(Decimal('91.399735'), Decimal('53.751351'))"
4,Anadyr (Carbon),2019,81.63,143.01,260.90,304.36,122.00,106.87,84.99,130.00,102.00,118.00,94.00,199.00,1746.76,"(Decimal('177.738273'), Decimal('64.713433'))"
5,Anapa (Vitjazevo),2019,45.92,53.15,54.00,54.72,52.00,67.45,172.31,72.57,70.00,63.00,69.00,82.10,856.22,"(Decimal('37.341511'), Decimal('45.003748'))"
8,Arkhangelsk (Talagy),2019,85.61,118.70,131.39,144.82,137.95,140.18,128.56,135.68,124.75,139.60,210.27,307.10,1804.61,"(Decimal('40.714892'), Decimal('64.596138'))"
9,Astrakhan (Narimanovo),2019,51.75,61.08,65.60,71.84,71.38,63.95,164.86,79.46,85.21,87.23,79.06,99.16,980.58,"(Decimal('47.999896'), Decimal('46.287344'))"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3693,Reads (tub),2007,55.96,80.09,85.90,154.54,162.71,107.51,80.14,138.71,133.19,188.97,228.84,184.00,1600.56,"(Decimal('113.306492'), Decimal('52.020464'))"
3705,Yuzhno-(Khomutovo),2007,710.80,970.00,1330.30,1352.30,1324.40,1613.00,1450.70,1815.60,1902.30,1903.20,1666.10,1632.10,17670.80,"(Decimal('142.723677'), Decimal('46.886967'))"
3706,Yakutsk,2007,583.70,707.80,851.80,1018.00,950.80,900.00,1154.90,1137.84,1485.50,1382.50,1488.00,1916.60,13577.44,"(Decimal('129.750225'), Decimal('62.086594'))"
3708,Yamburg,2007,3.55,0.16,3.37,5.32,4.31,6.30,6.88,3.60,4.13,4.93,4.17,8.87,55.59,"(Decimal('75.097783'), Decimal('67.980026'))"


#### (5) 采用一种合理的方式将所有机场划分为东南西北四个分区，并给出 2017年-2019 年货运总量最大的区域。

In [115]:
df['Airport coordinates'].values

array(["(Decimal('91.399735'), Decimal('53.751351'))",
       "(Decimal('111.543324'), Decimal('65.957161'))",
       "(Decimal('125.398355'), Decimal('58.602489'))", ...,
       "(Decimal('75.097783'), Decimal('67.980026'))",
       "(Decimal('40.170054'), Decimal('57.56231'))",
       "(Decimal('90.203907'), Decimal('60.252931'))"], dtype=object)

#### (6) 在统计学中常常用秩代表排名，现在规定某个机场某年某个月的秩为该机场该月在当年所有月份中货运量的排名（例如 *** 机场 19 年 1 月运量在整个 19 年 12 个月中排名第一，则秩为 1），那么判断某月运量情况的相对大小的秩方法为将所有机场在该月的秩排名相加，并将这个量定义为每一个月的秩综合指数，请根据上述定义计算 2016 年 12 个月的秩综合指数。

## 三、新冠肺炎在美国的传播
### 问题
#### (1) 用 corr() 函数计算县（每行都是一个县）人口与表中最后一天记录日期死亡数的相关系数。

In [9]:
import numpy as np
import pandas as pd

df_confirm = pd.read_csv('data/美国确证数.csv')
df_confirm.head()
df_death = pd.read_csv('data/美国死亡数.csv')
df_death.head()
df_death[['Population', '2020/4/26']].corr().iloc[0,1]

0.4038441973480568

#### (2) 截止到 4 月 1 日，统计每个州零感染县的比例。

In [28]:
def func(sub_df):
    #print('************')
    s_county_confirm_num = sub_df.drop(columns=list(sub_df.columns[:5])).sum(axis=1)
    return(s_county_confirm_num[s_county_confirm_num == 0].count() / s_county_confirm_num.count())
df_confirm.iloc[:, 6:-25].groupby('Province_State').apply(func)

Province_State
Alabama                 0.104478
Alaska                  0.793103
Arizona                 0.000000
Arkansas                0.293333
California              0.137931
Colorado                0.218750
Connecticut             0.000000
Delaware                0.000000
District of Columbia    0.000000
Florida                 0.164179
Georgia                 0.119497
Hawaii                  0.200000
Idaho                   0.386364
Illinois                0.450980
Indiana                 0.108696
Iowa                    0.404040
Kansas                  0.600000
Kentucky                0.433333
Louisiana               0.046875
Maine                   0.187500
Maryland                0.041667
Massachusetts           0.142857
Michigan                0.168675
Minnesota               0.356322
Mississippi             0.060976
Missouri                0.391304
Montana                 0.625000
Nebraska                0.741935
Nevada                  0.470588
New Hampshire           0.00

#### (3) 请找出最早出确证病例的三个县。

In [57]:
list_county = []
df_tmp = df_confirm.set_index('Admin2').iloc[:, 10:]
for column in df_tmp.columns:
    l_county = df_tmp[column][df_tmp[column] != 0].index.to_list()
    for i in l_county:
        if i not in list_county:
            list_county.append(i)
    if(len(list_county) >= 3):
        break
list_county[:3]

['King', 'Cook', 'Maricopa']

#### (4) 按州统计单日死亡增加数，并给出哪个州在哪一天确诊数增加最大（这里指的是在所有州和所有天两个指标一起算，不是分别算）。

In [96]:
def func(sub_df):
    #print('************')
    return(sub_df.iloc[:, 6:].sum())
df_death_in_day = df_death.iloc[:, 6:].groupby('Province_State').apply(func)
df_death_increase = pd.DataFrame()
df_death_increase['2020/1/22'] = df_death_in_day['2020/1/22']
pre_column = '2020/1/22'
for column in df_death_in_day.columns[1:]:
    df_death_increase[column] = df_death_in_day[column] - df_death_in_day[pre_column]
    pre_column = column
df_death_increase.head()

def func1(sub_df):
    #print('************')
    return(sub_df.iloc[:, 5:].sum())
df_confirm_in_day = df_confirm.iloc[:, 6:].groupby('Province_State').apply(func1)
df_confirm_increase = pd.DataFrame()
df_confirm_increase['2020/1/22'] = df_confirm_in_day['2020/1/22']
pre_column = '2020/1/22'
for column in df_confirm_in_day.columns[1:]:
    df_confirm_increase[column] = df_confirm_in_day[column] - df_confirm_in_day[pre_column]
    pre_column = column
df_confirm_increase.stack().idxmax()

('New York', '2020/4/15')

#### (5) 现需对每个州编制确证与死亡表，第一列为时间，并且起始时间为该州开始出现死亡比例的那一天，第二列和第三列分别为确证数和死亡数，每个州需要保存为一个单独的 csv 文件，文件名为“州名.csv”。

In [124]:
for province in df_confirm_in_day.index:
    df_province = pd.DataFrame()
    df_province['Date'] = df_confirm_in_day.columns
    df_province.set_index('Date', inplace=True)
    df_province['Confirm'] = df_confirm_in_day.loc[province]
    df_province['Death'] = df_death_in_day.loc[province]
    df_province.drop(df_province[df_province['Death'] == 0].index).reset_index().to_csv('output/{}.csv'.format(province), index=False)

In [104]:
df_confirm_in_day.head()

,2020/1/22,2020/1/23,2020/1/24,2020/1/25,2020/1/26,2020/1/27,2020/1/28,2020/1/29,2020/1/30,2020/1/31,...,2020/4/17,2020/4/18,2020/4/19,2020/4/20,2020/4/21,2020/4/22,2020/4/23,2020/4/24,2020/4/25,2020/4/26
Province_State,,,,,,,,,,,,,,,,,,,,,
Alabama,0,0,0,0,0,0,0,0,0,0,...,4571,4712,4888,5079,5317,5593,5832,6026,6026,6421
Alaska,0,0,0,0,0,0,0,0,0,0,...,309,314,319,321,329,335,337,339,339,340
Arizona,0,0,0,0,1,1,1,1,1,1,...,4511,4724,4933,5068,5256,5473,5772,6045,6286,6534
Arkansas,0,0,0,0,0,0,0,0,0,0,...,1652,1702,1735,1928,1945,2181,2472,2628,2709,2785
California,0,0,0,0,2,2,2,2,2,3,...,29157,30491,31431,33686,35465,37344,39561,41355,42368,43558


#### (6) 现需对 4 月 1 日至 4 月 10 日编制新增确证数与新增死亡数表，第一列为州名，第二列和第三列分别为新增确证数和新增死亡数，分别保存为十个单独的 csv 文件，文件名为“日期.csv”。

In [139]:
for date in df_confirm_increase.loc[:, '2020/4/1':'2020/4/10'].columns:
    df_date = pd.DataFrame()
    df_date['Province'] = df_confirm_in_day.index
    df_date.set_index('Province', inplace=True)
    df_date['ConfirmIncrease'] = df_confirm_increase[date]
    df_date['DeathIncrease'] = df_death_increase[date]
    df_date.reset_index().to_csv('output/{}.csv'.format(date.replace('/', '-')), index=False)

In [131]:
df_confirm_increase.head()

,2020/1/22,2020/1/23,2020/1/24,2020/1/25,2020/1/26,2020/1/27,2020/1/28,2020/1/29,2020/1/30,2020/1/31,...,2020/4/17,2020/4/18,2020/4/19,2020/4/20,2020/4/21,2020/4/22,2020/4/23,2020/4/24,2020/4/25,2020/4/26
Province_State,,,,,,,,,,,,,,,,,,,,,
Alabama,0,0,0,0,0,0,0,0,0,0,...,226,141,176,191,238,276,239,194,0,395
Alaska,0,0,0,0,0,0,0,0,0,0,...,9,5,5,2,8,6,2,2,0,1
Arizona,0,0,0,0,1,0,0,0,0,0,...,274,213,209,135,188,217,299,273,241,248
Arkansas,0,0,0,0,0,0,0,0,0,0,...,76,50,33,193,17,236,291,156,81,76
California,0,0,0,0,2,0,0,0,0,1,...,1480,1334,940,2255,1779,1879,2217,1794,1013,1190
